In [1]:
import glob
import os
import re
import pandas as pd
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from datetime import timedelta
import sys
import datetime
import tensorflow as tf
import math
from sklearn import cross_validation
import xgboost as xgb


/home/plexadmin/.virtualenvs/data-science-bowl-2017/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
OUTPUT_PATH = '/kaggle/dev/data-science-bowl-2017-data/submissions/'
DATA_PATH = '/kaggle/dev/data-science-bowl-2017-data/stage1_features/'
TENSORBOARD_SUMMARIES = '/kaggle/dev/data-science-bowl-2017-data/tensorboard_summaries/'
MODELS = '/kaggle_2/luna/luna16/models/'
MODEL_CHECKPOINTS = '/kaggle/dev/data-science-bowl-2017-data/models/checkpoints/'
MODEL_PATH = '/kaggle_2/luna/luna16/models/e03f0475-091e-4821-862e-ae5303d670c8/'
STAGE1 = '/kaggle/dev/data-science-bowl-2017-data/stage1/'
LABELS = '/kaggle/dev/data-science-bowl-2017-data/stage1_labels.csv'
STAGE1_SUBMISSION = '/kaggle/dev/data-science-bowl-2017-data/stage1_sample_submission.csv'
NAIVE_SUBMISSION = '/kaggle/dev/jovan/data-science-bowl-2017/data-science-bowl-2017/submissions/naive_submission.csv'

In [45]:
def get_inputs():
    num_chunks = 0

    inputs = {}
    labels = pd.read_csv(LABELS)
    input_features = {}

    for features in glob.glob(DATA_PATH + '*_transfer_values.npy')[0:10]:
        n = re.match('([a-f0-9].*)_transfer_values.npy', os.path.basename(features))
        patient_id = n.group(1)
        predictions = np.load(DATA_PATH + patient_id + '_predictions.npy')
        transfer_values = np.load(DATA_PATH + patient_id + '_transfer_values.npy')
        feature_val = np.append(predictions, transfer_values)
        try:
            label_val = int(labels.loc[labels['id'] == patient_id, 'cancer'])
        except TypeError:
            continue
        input_features[patient_id] = [np.append(predictions, transfer_values)]
        print('Patient {} predictions {} transfer_values {}'.format(patient_id, predictions.shape, transfer_values.shape))
    
    
    return 

get_inputs()

Patient 6ee742b62985570a1f3a142eb7e49188 predictions (5202, 7) transfer_values (5202, 512)
Patient d6d5ed3055d084a6abf0f97af3fe2ff0 predictions (6137, 7) transfer_values (6137, 512)
Patient 645e7f46eb9b834153ecf8e2b2921fe5 predictions (5832, 7) transfer_values (5832, 512)
Patient e6d8b2631843a24e6761f2723ea30788 predictions (6859, 7) transfer_values (6859, 512)
Patient f0f72264cd822301852578cc71288d3c predictions (5491, 7) transfer_values (5491, 512)
Patient a88c585e7d81744eec091a6f0600bd7b predictions (5184, 7) transfer_values (5184, 512)
Patient 39c3a2d2ca67bc7a1a22240ea571d50c predictions (6137, 7) transfer_values (6137, 512)
Patient 74b542d34b61740c1933d2c953403aa6 predictions (3840, 7) transfer_values (3840, 512)
Patient 174a9fc87f54d6def3730954fbafc99d predictions (3375, 7) transfer_values (3375, 512)
Patient 080e6a00e69888fd620894f9fd0714b1 predictions (7938, 7) transfer_values (7938, 512)
